# Test Your Fine-Tuned Symptom Extraction Model

This notebook loads your trained model and lets you test it with different patient descriptions.

---

## 1. Load Your Trained Model

In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np

# Load your fine-tuned model
print("🔄 Loading your fine-tuned model...")
model = SentenceTransformer('models/medical_symptom_matcher')
print("✅ Model loaded successfully!")

C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🔄 Loading your fine-tuned model...
✅ Model loaded successfully!


## 2. Load Symptoms Database

In [4]:
# Load the canonical symptoms
print("📋 Loading symptoms database...")
symptoms_df = pd.read_csv('data/symptoms.csv')
canonical_symptoms = symptoms_df['symptoms'].tolist()
print(f"✅ Loaded {len(canonical_symptoms)} symptoms")

# Show first few symptoms
print("\nFirst 10 symptoms:")
for i, symptom in enumerate(canonical_symptoms[:10], 1):
    print(f"  {i}. {symptom}")

📋 Loading symptoms database...
✅ Loaded 380 symptoms

First 10 symptoms:
  1. anxiety and nervousness
  2. depression
  3. shortness of breath
  4. depressive or psychotic symptoms
  5. sharp chest pain
  6. dizziness
  7. insomnia
  8. abnormal involuntary movements
  9. chest tightness
  10. palpitations


## 3. Pre-compute Symptom Embeddings

In [5]:
# Compute embeddings once (saves time when testing multiple inputs)
print("🧮 Computing symptom embeddings...")
symptom_embeddings = model.encode(canonical_symptoms, convert_to_tensor=True)
print(f"✅ Created embeddings with shape: {symptom_embeddings.shape}")

🧮 Computing symptom embeddings...
✅ Created embeddings with shape: torch.Size([380, 384])


## 4. Define Symptom Extraction Function

In [6]:
def extract_symptoms(patient_input, threshold=0.5, top_k=5):
    """
    Extract symptoms from patient's natural language description.
    
    Args:
        patient_input (str): What the patient says
        threshold (float): Minimum similarity score (0.0 to 1.0)
        top_k (int): Maximum number of symptoms to return
    
    Returns:
        list: Detected symptoms with confidence scores
    """
    # Encode the patient's input
    input_embedding = model.encode(patient_input, convert_to_tensor=True)
    
    # Calculate similarity with all symptoms
    similarities = util.cos_sim(input_embedding, symptom_embeddings)[0]
    
    # Get top matches above threshold
    top_results = []
    for idx, score in enumerate(similarities):
        if score >= threshold:
            top_results.append({
                'symptom': canonical_symptoms[idx],
                'confidence': float(score),
                'percentage': f"{float(score)*100:.1f}%"
            })
    
    # Sort by confidence and limit to top_k
    top_results = sorted(top_results, key=lambda x: x['confidence'], reverse=True)[:top_k]
    
    return top_results

def test_extraction(description, threshold=0.5):
    """Test the model and display results nicely."""
    print("="*70)
    print(f'Patient says: "{description}"')
    print("="*70)
    
    results = extract_symptoms(description, threshold=threshold)
    
    if results:
        print(f"\n✅ Detected {len(results)} symptom(s):\n")
        for i, result in enumerate(results, 1):
            print(f"  {i}. {result['symptom']}")
            print(f"     Confidence: {result['percentage']}")
            print()
    else:
        print("\n❌ No symptoms detected above threshold")
        print(f"   Try lowering threshold (currently {threshold})\n")
    
    return results

print("✅ Functions defined!")

✅ Functions defined!


## 5. Test Your Model!

In [7]:
# Test Case 1
test_extraction("I've been feeling dizzy and have a terrible headache")

Patient says: "I've been feeling dizzy and have a terrible headache"

✅ Detected 5 symptom(s):

  1. headache
     Confidence: 96.0%

  2. dizziness
     Confidence: 95.2%

  3. feeling ill
     Confidence: 94.7%

  4. fatigue
     Confidence: 93.3%

  5. ache all over
     Confidence: 92.2%



[{'symptom': 'headache',
  'confidence': 0.9600714445114136,
  'percentage': '96.0%'},
 {'symptom': 'dizziness',
  'confidence': 0.9519046545028687,
  'percentage': '95.2%'},
 {'symptom': 'feeling ill',
  'confidence': 0.9471486210823059,
  'percentage': '94.7%'},
 {'symptom': 'fatigue',
  'confidence': 0.9334315657615662,
  'percentage': '93.3%'},
 {'symptom': 'ache all over',
  'confidence': 0.9222030639648438,
  'percentage': '92.2%'}]

In [8]:
# Test Case 2
test_extraction("My throat is sore and I can't stop coughing")

Patient says: "My throat is sore and I can't stop coughing"

✅ Detected 5 symptom(s):

  1. sore throat
     Confidence: 95.9%

  2. throat irritation
     Confidence: 91.5%

  3. cough
     Confidence: 87.4%

  4. lump in throat
     Confidence: 86.5%

  5. sneezing
     Confidence: 85.4%



[{'symptom': 'sore throat',
  'confidence': 0.9593632221221924,
  'percentage': '95.9%'},
 {'symptom': 'throat irritation',
  'confidence': 0.9150927066802979,
  'percentage': '91.5%'},
 {'symptom': 'cough', 'confidence': 0.8738299012184143, 'percentage': '87.4%'},
 {'symptom': 'lump in throat',
  'confidence': 0.8649134039878845,
  'percentage': '86.5%'},
 {'symptom': 'sneezing',
  'confidence': 0.8539797067642212,
  'percentage': '85.4%'}]

In [9]:
# Test Case 3
test_extraction("I have chest pain and shortness of breath")

Patient says: "I have chest pain and shortness of breath"

✅ Detected 5 symptom(s):

  1. shortness of breath
     Confidence: 93.4%

  2. burning chest pain
     Confidence: 91.4%

  3. difficulty breathing
     Confidence: 88.2%

  4. chest tightness
     Confidence: 85.9%

  5. sharp chest pain
     Confidence: 84.9%



[{'symptom': 'shortness of breath',
  'confidence': 0.9336740374565125,
  'percentage': '93.4%'},
 {'symptom': 'burning chest pain',
  'confidence': 0.913640022277832,
  'percentage': '91.4%'},
 {'symptom': 'difficulty breathing',
  'confidence': 0.8817727565765381,
  'percentage': '88.2%'},
 {'symptom': 'chest tightness',
  'confidence': 0.8593178987503052,
  'percentage': '85.9%'},
 {'symptom': 'sharp chest pain',
  'confidence': 0.8490051031112671,
  'percentage': '84.9%'}]

In [10]:
# Test Case 4
test_extraction("I'm running a high fever and feeling nauseous")

Patient says: "I'm running a high fever and feeling nauseous"

✅ Detected 5 symptom(s):

  1. nausea
     Confidence: 93.8%

  2. headache
     Confidence: 89.6%

  3. feeling ill
     Confidence: 89.5%

  4. fatigue
     Confidence: 89.4%

  5. fever
     Confidence: 88.3%



[{'symptom': 'nausea',
  'confidence': 0.9380893111228943,
  'percentage': '93.8%'},
 {'symptom': 'headache',
  'confidence': 0.8961167335510254,
  'percentage': '89.6%'},
 {'symptom': 'feeling ill',
  'confidence': 0.8951557874679565,
  'percentage': '89.5%'},
 {'symptom': 'fatigue',
  'confidence': 0.8943477869033813,
  'percentage': '89.4%'},
 {'symptom': 'fever', 'confidence': 0.8829719424247742, 'percentage': '88.3%'}]

In [12]:
# Test Case 5
test_extraction("My stomach hurts and I have diarrhea")

Patient says: "My stomach hurts and I have diarrhea"

✅ Detected 5 symptom(s):

  1. diarrhea
     Confidence: 96.2%

  2. blood in stool
     Confidence: 93.2%

  3. constipation
     Confidence: 93.0%

  4. stomach bloating
     Confidence: 82.9%

  5. jaundice
     Confidence: 78.9%



[{'symptom': 'diarrhea',
  'confidence': 0.962323784828186,
  'percentage': '96.2%'},
 {'symptom': 'blood in stool',
  'confidence': 0.9320855736732483,
  'percentage': '93.2%'},
 {'symptom': 'constipation',
  'confidence': 0.9298596382141113,
  'percentage': '93.0%'},
 {'symptom': 'stomach bloating',
  'confidence': 0.8285396099090576,
  'percentage': '82.9%'},
 {'symptom': 'jaundice',
  'confidence': 0.7885253429412842,
  'percentage': '78.9%'}]

## 6. Try Your Own!

Modify the text below to test with your own patient descriptions:

In [18]:
# Try your own description here!
my_description = "I saw some reddish rash appear on my leg, is this ezcema"

test_extraction(my_description, threshold=0.5)

Patient says: "I saw some reddish rash appear on my leg, is this ezcema"

✅ Detected 5 symptom(s):

  1. skin rash
     Confidence: 85.3%

  2. itching of skin
     Confidence: 84.1%

  3. skin lesion
     Confidence: 82.9%

  4. rough skin
     Confidence: 79.9%

  5. skin irritation
     Confidence: 77.8%



[{'symptom': 'skin rash',
  'confidence': 0.8530450463294983,
  'percentage': '85.3%'},
 {'symptom': 'itching of skin',
  'confidence': 0.8411623239517212,
  'percentage': '84.1%'},
 {'symptom': 'skin lesion',
  'confidence': 0.8292899131774902,
  'percentage': '82.9%'},
 {'symptom': 'rough skin',
  'confidence': 0.7988935708999634,
  'percentage': '79.9%'},
 {'symptom': 'skin irritation',
  'confidence': 0.778387188911438,
  'percentage': '77.8%'}]

## 7. Adjust Threshold

Lower threshold = more symptoms detected (but less confident)  
Higher threshold = fewer symptoms (but more confident)

In [15]:
# Test with different thresholds
description = "I feel really tired and weak"

print("Testing with different thresholds:\n")

for threshold in [0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f"\nThreshold = {threshold}")
    print("-" * 40)
    results = extract_symptoms(description, threshold=threshold)
    if results:
        for r in results:
            print(f"  • {r['symptom']} ({r['percentage']})")
    else:
        print("  (no symptoms detected)")

Testing with different thresholds:


Threshold = 0.3
----------------------------------------
  • fatigue (93.1%)
  • dizziness (91.2%)
  • weakness (91.0%)
  • shortness of breath (89.1%)
  • back weakness (88.2%)

Threshold = 0.4
----------------------------------------
  • fatigue (93.1%)
  • dizziness (91.2%)
  • weakness (91.0%)
  • shortness of breath (89.1%)
  • back weakness (88.2%)

Threshold = 0.5
----------------------------------------
  • fatigue (93.1%)
  • dizziness (91.2%)
  • weakness (91.0%)
  • shortness of breath (89.1%)
  • back weakness (88.2%)

Threshold = 0.6
----------------------------------------
  • fatigue (93.1%)
  • dizziness (91.2%)
  • weakness (91.0%)
  • shortness of breath (89.1%)
  • back weakness (88.2%)

Threshold = 0.7
----------------------------------------
  • fatigue (93.1%)
  • dizziness (91.2%)
  • weakness (91.0%)
  • shortness of breath (89.1%)
  • back weakness (88.2%)


## 8. Batch Testing

Test multiple descriptions at once:

In [16]:
test_cases = [
    "I can't breathe properly",
    "My joints are aching",
    "I feel like throwing up",
    "My vision is blurry",
    "I have a rash on my arms"
]

for test_case in test_cases:
    test_extraction(test_case)
    print("\n")

Patient says: "I can't breathe properly"

✅ Detected 5 symptom(s):

  1. difficulty breathing
     Confidence: 97.8%

  2. breathing fast
     Confidence: 93.4%

  3. shortness of breath
     Confidence: 91.3%

  4. hurts to breath
     Confidence: 90.2%

  5. wheezing
     Confidence: 89.5%



Patient says: "My joints are aching"

✅ Detected 5 symptom(s):

  1. knee pain
     Confidence: 93.9%

  2. joint swelling
     Confidence: 93.2%

  3. leg swelling
     Confidence: 91.6%

  4. knee swelling
     Confidence: 89.6%

  5. elbow pain
     Confidence: 89.1%



Patient says: "I feel like throwing up"

✅ Detected 5 symptom(s):

  1. nausea
     Confidence: 84.4%

  2. feeling ill
     Confidence: 83.1%

  3. weight gain
     Confidence: 81.4%

  4. headache
     Confidence: 80.9%

  5. fever
     Confidence: 80.9%



Patient says: "My vision is blurry"

✅ Detected 5 symptom(s):

  1. double vision
     Confidence: 93.0%

  2. eye redness
     Confidence: 84.1%

  3. blindness
     Con

---
## 🎉 Done!

### Next Steps:
- Integrate this into your chatbot
- Adjust `threshold` based on your needs
- Add more symptoms to your database if needed

### Tips:
- **Lower threshold (0.3-0.4)**: More sensitive, catches more symptoms
- **Higher threshold (0.6-0.7)**: More precise, only high-confidence matches